# Free Response Answer Type

This notebook demonstrates how to generate open-ended questions with free-form text answers. Free response questions are perfect for open-ended analysis, explanation generation, and creative forecasting where you need detailed, nuanced answers.

In [1]:
%pip install -e ..
%pip install python-dotenv

from IPython.display import clear_output
clear_output()

## Set up the client

**Set your API key:**

- **Locally**: Create a `.env` file in this directory with `LIGHTNINGROD_API_KEY=your_key_here`
- **Google Colab**: Go to the Secrets section (🔑 icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`

In [2]:
from utils import get_secret
from lightningrod import LightningRod

api_key = get_secret("LIGHTNINGROD_API_KEY")
base_url = get_secret("LIGHTNINGROD_BASE_URL", "https://api.lightningrod.ai/api/public/v1")

lr = LightningRod(api_key=api_key, base_url=base_url)

## Free Response Answer Type Overview

Free response questions return full text answers, not just single values or choices. They're ideal for:
- Open-ended analysis and explanations
- Creative forecasting
- Detailed scenario descriptions
- Questions that require nuanced, multi-sentence answers

Free response questions work with any data source: Google News, GDELT, or custom documents.

In [3]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator, QuestionGenerator, WebSearchLabeler, QuestionRenderer, QuestionPipeline

answer_type = AnswerType(answer_type=AnswerTypeEnum.FREE_RESPONSE)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query="technology industry trends",
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate open-ended forecasting questions that require detailed explanations. "
        "Questions should ask for analysis, predictions, or explanations that require multiple sentences to answer."
    ),
    examples=[
        "What will be the main theme of Apple's next product launch?",
        "Explain the potential impact of the new AI regulation on tech companies.",
        "What trends will dominate the technology industry in 2025?",
        "Describe how the recent merger will affect the competitive landscape.",
    ],
    bad_examples=[
        "Will Apple release a new product?",
        "What is AI?",
        "When will the product launch?",
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=QuestionRenderer(answer_type=answer_type),
)

## Run the Pipeline

Generate free response questions from the news articles.

In [4]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [5]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [6]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df.head()

Generated 10 samples



,question.question_text,label.label,label.label_confidence,label.reasoning,label.answer_sources,prompt,seed.seed_text,seed.url,seed.seed_creation_date,seed.search_query,is_valid,meta.sample_id,meta.parent_sample_id,meta.processing_time_ms,meta.filter_reason
0,Analyze how the four quantum computing trends ...,"By the end of the decade, Quandela's identifie...",0.9,The four quantum computing trends identified b...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nAnalyze how the four quantum comput...,Title: Quandela Identifies Four Quantum Comput...,https://thequantuminsider.com/2026/01/15/quand...,2026-01-14T00:00:00,technology industry trends,True,4500d39b-877f-4dd3-8c7c-bf60011bddfc,83fae696-b04d-4ae0-b413-dc4d46da3aa3,41440.826,NaN
1,"Based on the trends identified for 2026, how w...",The convergence of an 'AI Backbone' and 'Cloud...,0.9,The provided text comprehensively details how ...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nBased on the trends identified for ...,Title: Top Tech Trends of 2026\n\nURL Source: ...,https://www.capgemini.com/insights/research-li...,2026-01-14T00:00:00,technology industry trends,True,80c9fcd6-1ece-4e75-8b83-b918e56e951a,47ee23c1-72c9-4a15-b3ce-6115fb34627f,24401.795,NaN
2,Analyze how the strategic trajectories of Micr...,undetermined,1.0,The analysis of the provided search results in...,https://vertexaisearch.cloud.google.com/ground...,NaN,Title: Zacks Industry Outlook Highlights Micro...,https://finance.yahoo.com/news/zacks-industry-...,2026-01-14T00:00:00,technology industry trends,False,8bb91952-c3b1-4f8c-8c5b-8d6c7746f8cc,fa934282-3dbc-4121-8c3f-387fda5746ad,31317.680,Undetermined label
3,Considering the viewpoints of leaders from Cir...,The shift toward an 'agentic workforce' and th...,0.9,The answer is based on multiple recent search ...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nConsidering the viewpoints of leade...,Title: The future of jobs: 6 decision-makers o...,https://www.weforum.org/stories/2026/01/how-ai...,2026-01-14T00:00:00,technology industry trends,True,38fa60aa-1692-41fc-b0f3-7fa2bea4153d,2ef12a2a-17b0-4b98-ba8c-f1a1f17e42fa,30821.616,NaN
4,Based on the predicted transition from keyword...,"By 2027, the role of legal professionals is pr...",0.9,The various search results consistently outlin...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nBased on the predicted transition f...,Title: 2026 Legal Tech Trends | JD Supra\n\nUR...,https://www.jdsupra.com/legalnews/2026-legal-t...,2026-01-21T00:00:00,technology industry trends,True,426ddc0f-18e2-4041-9995-4aefa06c4d0a,53df32ce-6ca6-422c-bd5a-36d909e59c71,18035.059,NaN


## Use Cases for Free Response Questions

**Open-Ended Analysis:**
- "Explain the impact of X on Y"
- "What are the implications of Z?"

**Creative Forecasting:**
- "What will be the main theme of the next product launch?"
- "Describe how the industry will evolve"

**Detailed Predictions:**
- "What trends will dominate?"
- "How will the market change?"